In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from fuzzywuzzy import fuzz

# Load the CSV file
df = pd.read_csv('all_test_predicted_after_parse_final.csv')

df["Rewritten_Parse"] = df["Rewritten_Parse"].str.replace('\n ', '')
df["Predicted_Parse"] = df["Predicted_Parse"].str.replace('\n ', '')
df["Rewritten_Parse"] = df["Rewritten_Parse"].str.replace('\n', '')
df["Predicted_Parse"] = df["Predicted_Parse"].str.replace('\n', '')
df["Rewritten_Parse"] = df["Rewritten_Parse"].str.replace('```', '')
df["Predicted_Parse"] = df["Predicted_Parse"].str.replace('```', '')
df["Rewritten_Parse"] = df["Rewritten_Parse"].str.replace('json', '')
df["Predicted_Parse"] = df["Predicted_Parse"].str.replace('json', '')
df["Rewritten_Parse"] = df["Rewritten_Parse"].str.lower()
df["Predicted_Parse"] = df["Predicted_Parse"].str.lower()
df

In [ ]:
def compute_fuzzy_match_accuracy(expected_args, predicted_args):
    fuzzy_scores = {}
    for key in expected_args:
        if key in predicted_args:
            expected_value = str(expected_args[key])  # Ensure value is converted to string
            predicted_value = str(predicted_args[key])  # Ensure value is converted to string

            # Check for empty values or None
            if not expected_value or not predicted_value:
                fuzzy_scores[key] = 0  # Assign 0 score for empty or None values
            else:
                ratio = fuzz.ratio(expected_value, predicted_value)
                fuzzy_scores[key] = ratio

    total_keys = len(fuzzy_scores)
    total_accuracy = sum(fuzzy_scores.values()) / total_keys if total_keys > 0 else 0
    return total_accuracy, fuzzy_scores
import json

# Initialize lists to store intent and argument data
expected_intents = []
predicted_intents = []
fuzzy_accuracy_scores = []

# Iterate over rows of the DataFrame
for index, row in df.iterrows():
    expected_parse = json.loads(row["Rewritten_Parse"])
    predicted_parse = json.loads(row["Predicted_Parse"])

    # Extracting intent from both expected and predicted parses
    expected_intent = expected_parse["intent"]
    predicted_intent = predicted_parse["intent"]

    # Append intent data to lists
    expected_intents.append(expected_intent)
    predicted_intents.append(predicted_intent)

    # Compute fuzzy match accuracy for the argument field
    expected_args = expected_parse.get("arguments", {})
    predicted_args = predicted_parse.get("arguments", {})
    
    # Call the function to compute fuzzy match accuracy
    accuracy, _ = compute_fuzzy_match_accuracy(expected_args, predicted_args)
    fuzzy_accuracy_scores.append(accuracy)

# Overall classification report for intent
overall_classification_report = classification_report(expected_intents, predicted_intents)

# Overall fuzzy match accuracy for argument fields
if len(fuzzy_accuracy_scores) > 0:
    overall_fuzzy_accuracy = sum(fuzzy_accuracy_scores) / len(fuzzy_accuracy_scores)
else:
    overall_fuzzy_accuracy = 0.0

print("Overall Classification Report for 'Intent' Field:")
print(overall_classification_report)

print("\nOverall Fuzzy Match Accuracy for Argument Fields:")
print(f"Overall Accuracy: {overall_fuzzy_accuracy:.2f}")